# Desarrollo de DF único - Sample

## Preprocesamiento

In [2]:
import stylo_metrix as sm
import pandas as pd
from raid import run_detection, run_evaluation
from raid.utils import load_data

from processer import split_text_into_sentences
from sklearn.preprocessing import LabelEncoder

In [3]:
# Download the RAID dataset without adversarial attacks
or_train_noadv_df = load_data(split="train", include_adversarial=False)

In [4]:
print(f"Original training data shape: {or_train_noadv_df.shape}")

Original training data shape: (467985, 11)


In [5]:
intCols = ['id','model', 'domain', 'title', 'prompt', 'generation']

# Copia del dataframe con columnas específicas
train_noadv_df = or_train_noadv_df.copy()
train_noadv_df = train_noadv_df[intCols]

## Versión 1 - Fusión original

In [6]:
filtered_by_domain = train_noadv_df[
    (train_noadv_df['domain'] != 'recipes')
    ]

# Separar documentos humanos y de IA
human_docs = filtered_by_domain[filtered_by_domain['model'] == 'human']
ai_docs = filtered_by_domain[filtered_by_domain['model'] != 'human']

total_samples = 1000
samples_per_class = total_samples // 2  # 500 de cada uno

# Samplear 50% de cada clase
human_sample = human_docs[['id', 'model', 'domain', 'generation']].sample(n=samples_per_class, random_state=50)
ai_sample = ai_docs[['id', 'model', 'domain', 'generation']].sample(n=samples_per_class, random_state=50)

# Combinar ambos samples
generation_sample = pd.concat([human_sample, ai_sample], ignore_index=True)

generation_sample = generation_sample.sample(frac=1, random_state=50).reset_index(drop=True)

print(f"Total muestras: {len(generation_sample)}")
print(f"Distribución por clase:")
print(generation_sample['model'].value_counts())
print(f"\nHumanos: {(generation_sample['model'] == 'human').sum()} ({(generation_sample['model'] == 'human').sum() / len(generation_sample) * 100:.1f}%)")
print(f"IA: {(generation_sample['model'] != 'human').sum()} ({(generation_sample['model'] != 'human').sum() / len(generation_sample) * 100:.1f}%)")

Total muestras: 1000
Distribución por clase:
model
human           500
gpt2             64
mistral-chat     62
mpt-chat         61
mpt              58
mistral          54
llama-chat       50
chatgpt          35
gpt3             34
cohere           31
cohere-chat      27
gpt4             24
Name: count, dtype: int64

Humanos: 500 (50.0%)
IA: 500 (50.0%)


In [7]:
display(generation_sample)

,id,model,domain,generation
0,3ef6e59b-f260-40eb-954d-9a8afb720bd3,human,abstracts,Image-to-image translation is a long-establish...
1,c0161712-fde5-42e2-adf3-7af56ca33f03,human,abstracts,"We like to develop model theory for $T$, a c..."
2,5388ced8-10ce-46f0-b6cb-95de870a94e9,mpt,abstracts,"In this work, we propose an image processing m..."
3,84e3178a-b8b3-441b-b1be-ffd75e86608a,human,reddit,The amount of frustrated users I see on every ...
4,926582e6-b145-425a-bf83-31e6428d6a40,mpt-chat,books,"In the small town of Ashwood, people are myste..."
...,...,...,...,...
995,008eb9c5-3a4f-4f3f-b989-039ab81f17a8,human,books,Lincoln Rhyme is away from the city with his ...
996,7411281e-70bd-4d4f-b20e-39742ef83c63,human,reddit,I am really tired of people ignoring me when I...
997,1b70d305-81de-46ef-bd8b-4e983f93bc08,llama-chat,books,"Sure, here's a plot summary for a novel titled..."
998,7f2d89d1-bfb6-4262-9a03-f52305af5430,human,books,"The hero is Matthew Dodd, a rifleman in the 9..."


In [8]:
def extract_features_from_dataset(df_original, sample_size=None):
    """
    Extrae features estilométricos a nivel de oración.
    
    Returns:
        DataFrame con estructura: id_original, model, domain, sentence_num, text, features...
    """
    if sample_size:
        df_original = df_original.sample(n=sample_size, random_state=42)
    
    # Inicializar StyloMetrix (sin guardar archivos)
    stylo = sm.StyloMetrix('en', debug=False)  # debug=False para evitar archivos
    
    all_results = []
    
    for idx, row in df_original.iterrows():
        # Dividir en oraciones (en memoria)
        sentences = split_text_into_sentences(row['generation'])
        
        # Extraer features para todas las oraciones del documento
        features_df = stylo.transform(sentences)
        
        # Agregar metadatos del documento original
        features_df.insert(0, 'id_original', row['id'])
        features_df.insert(1, 'model', row['model'])
        features_df.insert(2, 'domain', row['domain'])
        features_df.insert(3, 'sentence_num', range(len(sentences)))
        # La columna 'text' ya existe en features_df (viene de stylo.transform)
        
        all_results.append(features_df)
    
    # Concatenar todos los resultados
    final_df = pd.concat(all_results, ignore_index=True)
    
    return final_df

In [9]:
features_df = extract_features_from_dataset(generation_sample)

c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_trf' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


[OK] Total de oraciones: 8


  0%|          | 0/8 [00:00<?, ?it/s]c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
100%|██████████| 8/8 [00:01<00:00,  4.10it/s]


[OK] Total de oraciones: 7


  0%|          | 0/7 [00:00<?, ?it/s]c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
100%|██████████| 7/7 [00:01<00:00,  4.01it/s]


...

  0%|          | 0/20 [00:00<?, ?it/s]c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
100%|██████████| 20/20 [00:02<00:00,  7.50it/s]


[OK] Total de oraciones: 29


  0%|          | 0/29 [00:00<?, ?it/s]c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
100%|██████████| 29/29 [00:05<00:00,  5.71it/s]


[OK] Total de oraciones: 3


  0%|          | 0/3 [00:00<?, ?it/s]c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
100%|██████████| 3/3 [00:02<00:00,  1.22it/s]


In [10]:
# Uso:
print(f"Shape: {features_df.shape}")
display(features_df.head(5))

Shape: (12168, 201)


,id_original,model,domain,sentence_num,text,POS_VERB,POS_NOUN,POS_ADJ,POS_ADV,POS_DET,...,RE,ASF,ASM,OM,RCI,DMC,OR,QAS,PA,PR
0,3ef6e59b-f260-40eb-954d-9a8afb720bd3,human,abstracts,0,Image-to-image translation is a long-establish...,0.166667,0.500000,0.083333,0.083333,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
1,3ef6e59b-f260-40eb-954d-9a8afb720bd3,human,abstracts,1,In this paper we propose an adversarial based ...,0.166667,0.500000,0.000000,0.000000,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.000000,0.0,0.0
2,3ef6e59b-f260-40eb-954d-9a8afb720bd3,human,abstracts,2,The regular deep neural-network based methods ...,0.208333,0.500000,0.125000,0.000000,0.083333,...,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,0.083333,0.0,0.0
3,3ef6e59b-f260-40eb-954d-9a8afb720bd3,human,abstracts,3,Our generative adversarial network based model...,0.166667,0.333333,0.250000,0.000000,0.083333,...,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.000000,0.0,0.0
4,3ef6e59b-f260-40eb-954d-9a8afb720bd3,human,abstracts,4,This approach makes the image translation inde...,0.062500,0.375000,0.187500,0.000000,0.187500,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.125000,0.0,0.0


## Versión 2 - Tags codificados

### Codificación de etiquetas 'model' y 'domain'

In [11]:
print("Información del dataset original:")
print(f"Forma del dataset: {or_train_noadv_df.shape}")
print(f"Columnas: {list(or_train_noadv_df.columns)}")
print(f"Modelos unicos: {or_train_noadv_df['model'].unique()}")
print(f"Dominios unicos: {or_train_noadv_df['domain'].unique()}")

Información del dataset original:
Forma del dataset: (467985, 11)
Columnas: ['id', 'adv_source_id', 'source_id', 'model', 'decoding', 'repetition_penalty', 'attack', 'domain', 'title', 'prompt', 'generation']
Modelos unicos: ['human' 'llama-chat' 'mpt' 'mpt-chat' 'gpt2' 'mistral' 'mistral-chat'
 'gpt3' 'cohere' 'chatgpt' 'gpt4' 'cohere-chat']
Dominios unicos: ['abstracts' 'books' 'news' 'poetry' 'recipes' 'reddit' 'reviews' 'wiki']


In [12]:
print("Información del dataset generado:")
print(f"Forma del dataset: {features_df.shape}")
print(f"Columnas: {list(features_df.columns)}")
print(f"Modelos unicos: {features_df['model'].unique()}")
print(f"Dominios unicos: {features_df['domain'].unique()}")

Información del dataset generado:
Forma del dataset: (12168, 201)
Columnas: ['id_original', 'model', 'domain', 'sentence_num', 'text', 'POS_VERB', 'POS_NOUN', 'POS_ADJ', 'POS_ADV', 'POS_DET', 'POS_INTJ', 'POS_CONJ', 'POS_PART', 'POS_NUM', 'POS_PREP', 'POS_PRO', 'L_REF', 'L_HASHTAG', 'L_MENTION', 'L_RT', 'L_LINKS', 'L_CONT_A', 'L_FUNC_A', 'L_CONT_T', 'L_FUNC_T', 'L_PLURAL_NOUNS', 'L_SINGULAR_NOUNS', 'L_PROPER_NAME', 'L_PERSONAL_NAME', 'L_NOUN_PHRASES', 'L_PUNCT', 'L_PUNCT_DOT', 'L_PUNCT_COM', 'L_PUNCT_SEMC', 'L_PUNCT_COL', 'L_PUNCT_DASH', 'L_POSSESSIVES', 'L_ADJ_POSITIVE', 'L_ADJ_COMPARATIVE', 'L_ADJ_SUPERLATIVE', 'L_ADV_POSITIVE', 'L_ADV_COMPARATIVE', 'L_ADV_SUPERLATIVE', 'PS_CONTRADICTION', 'PS_AGREEMENT', 'PS_EXAMPLES', 'PS_CONSEQUENCE', 'PS_CAUSE', 'PS_LOCATION', 'PS_TIME', 'PS_CONDITION', 'PS_MANNER', 'SY_QUESTION', 'SY_NARRATIVE', 'SY_NEGATIVE_QUESTIONS', 'SY_SPECIAL_QUESTIONS', 'SY_TAG_QUESTIONS', 'SY_GENERAL_QUESTIONS', 'SY_EXCLAMATION', 'SY_IMPERATIVE', 'SY_SUBORD_SENT', 'SY_SU

In [13]:
# Codificación de etiquetas 'model' y 'domain'
id_encoder = LabelEncoder()
features_df['id_encoded'] = id_encoder.fit_transform(features_df['id_original'])

In [24]:
# Visualizar texto de registros aleatorios de dominio poetry
pd.set_option('display.max_colwidth', None)  # Sin límite de ancho
pd.set_option('display.max_rows', None)      # Sin límite de filas (usar con cuidado)

poetry_df = features_df[features_df['domain'] == 'poetry']
display(poetry_df[['id_original', 'sentence_num', 'model', 'domain', 'text']].sample(n=5, random_state=11))

pd.reset_option('display.max_colwidth')
pd.reset_option('display.max_rows')

,id_original,sentence_num,model,domain,text
5357,c8b310ff-8203-4f90-9bf6-5ac6c82b1819,3,human,poetry,"A dim desire, of pleasant places, And lush fields in the summer sun, And logs aflame, and walls, and faces, -- And wine, and old ambrosial talk, A golden ball in fountains dancing, And unforgotten hands. (Ah, God, I trod them down where I have trod, And they remain, and they remain, Etched in unutterable pain, Loved lips and faces now apart, That once were closer than my heart -- In agony, in agony, And horribly a part of me. . . ."
6148,d982e029-6184-489b-9251-37ad6ccd1a85,8,mpt,poetry,"This is the best interpretation of nature that man will ever find, And this is nature as it is for man had not yet set foot on nature's ground."
2619,ba673ef7-a429-4bfb-9d50-8486653e5a34,344,human,poetry,"Here, where the walls go down beneath our picks, These walls whose windows gap against the sky, Atom by atom of flesh and brain and marble Will build a glittering tower before we die . . ."
10364,2fb69dc2-6490-4ab2-a13a-d187b27efb95,8,gpt2,poetry,"So, I decided to start writing down some things that trigger me."
9844,23ca752c-298d-44d1-b933-a0f16690619f,3,human,poetry,We are the Little Folk--we!


In [ ]:
# Análisis de documentos con una sola oración
# Estos casos suelen indicar problemas en el split de oraciones

# Contar oraciones por ID
oraciones_por_id = features_df.groupby('id_encoded')['sentence_num'].apply(lambda x: x.tolist())

# Filtrar IDs que solo tienen sentence_num = 0
ids_una_oracion = oraciones_por_id[oraciones_por_id.apply(lambda x: x == [0])].index

print(f"ANÁLISIS DE DOCUMENTOS CON UNA SOLA ORACIÓN")
print(f"{'='*60}")
print(f"Documentos con solo 1 oración: {len(ids_una_oracion)}")
print(f"Total de documentos: {features_df['id_encoded'].nunique()}")
print(f"Porcentaje: {len(ids_una_oracion) / features_df['id_encoded'].nunique() * 100:.2f}%")

# Analizar distribución por dominio
print(f"\nDistribución por dominio:")
df_una_oracion_temp = features_df[features_df['id_encoded'].isin(ids_una_oracion)]
print(df_una_oracion_temp.groupby('domain')['id_encoded'].nunique())

# Analizar distribución por modelo (human vs IA)
print(f"\nDistribución por modelo:")
print(df_una_oracion_temp.groupby('model')['id_encoded'].nunique())

#### Opciones de filtrado

Basado en el análisis anterior, decide qué estrategia seguir:
- **Opción 1**: Excluir solo documentos con 1 oración
- **Opción 2**: Excluir dominio 'poetry' completo (recomendado si poetry tiene muchos problemas)
- **Opción 3**: Excluir ambos (más conservador, dataset más limpio)

In [ ]:
# CONFIGURACIÓN: Elige tu estrategia de filtrado
# ================================================

# Opción 1: Solo excluir documentos con 1 oración
EXCLUIR_UNA_ORACION = True

# Opción 2: Excluir dominio poetry completo
EXCLUIR_POETRY = False

# Opción 3: Excluir documentos de poetry con 1 oración (intermedio)
EXCLUIR_POETRY_UNA_ORACION = False

# ================================================
# Aplicar filtros
# ================================================

features_df_filtrado = features_df.copy()

# Obtener IDs a excluir
ids_a_excluir = set()

if EXCLUIR_UNA_ORACION:
    ids_a_excluir.update(ids_una_oracion)
    print(f"✓ Excluyendo documentos con 1 sola oración: {len(ids_una_oracion)} IDs")

if EXCLUIR_POETRY:
    ids_poetry = features_df[features_df['domain'] == 'poetry']['id_encoded'].unique()
    ids_a_excluir.update(ids_poetry)
    print(f"✓ Excluyendo todo el dominio poetry: {len(ids_poetry)} IDs")

if EXCLUIR_POETRY_UNA_ORACION:
    ids_poetry_una = features_df[
        (features_df['domain'] == 'poetry') & 
        (features_df['id_encoded'].isin(ids_una_oracion))
    ]['id_encoded'].unique()
    ids_a_excluir.update(ids_poetry_una)
    print(f"✓ Excluyendo poetry con 1 oración: {len(ids_poetry_una)} IDs")

# Aplicar exclusiones
if ids_a_excluir:
    features_df_filtrado = features_df_filtrado[~features_df_filtrado['id_encoded'].isin(ids_a_excluir)]
    
    print(f"\n{'='*60}")
    print(f"📊 RESULTADO DEL FILTRADO:")
    print(f"{'='*60}")
    print(f"IDs excluidos: {len(ids_a_excluir)}")
    print(f"IDs restantes: {features_df_filtrado['id_encoded'].nunique()}")
    print(f"Registros antes: {len(features_df)}")
    print(f"Registros después: {len(features_df_filtrado)}")
    print(f"Reducción: {((len(features_df) - len(features_df_filtrado)) / len(features_df) * 100):.2f}%")
    
    print(f"\n🎯 Distribución final por modelo:")
    print(features_df_filtrado.groupby('model')['id_encoded'].nunique())
    
    print(f"\n🌍 Distribución final por dominio:")
    print(features_df_filtrado.groupby('domain')['id_encoded'].nunique())
else:
    print("⚠️ No se aplicaron filtros")

In [ ]:
# APLICAR EL FILTRADO (ejecuta esta celda solo si estás conforme con los resultados)
# ===================================================================================

# Reemplazar features_df con la versión filtrada
features_df = features_df_filtrado.copy()

print("✅ DataFrame filtrado aplicado exitosamente")
print(f"Documentos únicos: {features_df['id_encoded'].nunique()}")
print(f"Total de registros (oraciones): {len(features_df)}")

#### Control de calidad: Análisis y filtrado de datos problemáticos

**Problema identificado:** La función `split_text_into_sentences()` tiene limitaciones:
- Requiere patrón: `[.!?] + espacio + Mayúscula`
- Falla en textos con puntuación irregular
- Especialmente problemático en poetry (sin puntuación tradicional)

**Solución:** Excluir documentos problemáticos para mantener consistencia del dataset.

**Ventajas de excluir vs. mejorar split:**
1. ✅ Mantiene consistencia (todos procesados con mismo método)
2. ✅ Evita rehacer todo el procesamiento 
3. ✅ Dataset más limpio y confiable
4. ✅ Mejor generalización del modelo

In [3]:
# Ordenar DF de visualización de texto
vis_df = features_df.copy()

visCols = ['id_encoded', 'sentence_num', 'model', 'text', 'domain']

vis_df = vis_df[visCols]

vis_df = vis_df.rename(columns={
    'id_encoded': 'id',
    # 'model_encoded': 'model_label',
    # 'domain_encoded': 'domain_label'
})

vis_df = vis_df.sort_values(by=['id', 'sentence_num']).reset_index(drop=True)
display(vis_df.head(10))

NameError: name 'features_df' is not defined

In [2]:
pd.set_option('display.max_colwidth', None)  # Sin límite de ancho
pd.set_option('display.max_rows', None)      # Sin límite de filas (usar con cuidado)


# Verificar documentos con solo una oración (sentence_num = 0 únicamente)
# Contar oraciones por ID
oraciones_por_id = vis_df.groupby('id')['sentence_num'].apply(lambda x: x.tolist())

# Filtrar IDs que solo tienen sentence_num = 0
ids_una_oracion = oraciones_por_id[oraciones_por_id.apply(lambda x: x == [0])].index

print(f"Número de documentos con solo una oración: {len(ids_una_oracion)}")
print(f"Total de documentos: {vis_df['id'].nunique()}")
print(f"Porcentaje: {len(ids_una_oracion) / vis_df['id'].nunique() * 100:.2f}%")

# Mostrar df con los documentos con solo una oración
df_una_oracion = vis_df[vis_df['id'].isin(ids_una_oracion)]
display(df_una_oracion)


pd.reset_option('display.max_colwidth')
pd.reset_option('display.max_rows')

NameError: name 'pd' is not defined

### Armado de DF

In [32]:
# Ordenar DF final
train_df = features_df.copy()

trainCols = ['id_encoded', 'sentence_num', 'model', 'domain', 'POS_VERB', 'POS_NOUN', 'POS_ADJ', 'POS_ADV', 'POS_DET', 'POS_INTJ', 'POS_CONJ', 'POS_PART', 'POS_NUM', 'POS_PREP', 'POS_PRO', 'L_REF', 'L_HASHTAG', 'L_MENTION', 'L_RT', 'L_LINKS', 'L_CONT_A', 'L_FUNC_A', 'L_CONT_T', 'L_FUNC_T', 'L_PLURAL_NOUNS', 'L_SINGULAR_NOUNS', 'L_PROPER_NAME', 'L_PERSONAL_NAME', 'L_NOUN_PHRASES', 'L_PUNCT', 'L_PUNCT_DOT', 'L_PUNCT_COM', 'L_PUNCT_SEMC', 'L_PUNCT_COL', 'L_PUNCT_DASH', 'L_POSSESSIVES', 'L_ADJ_POSITIVE', 'L_ADJ_COMPARATIVE', 'L_ADJ_SUPERLATIVE', 'L_ADV_POSITIVE', 'L_ADV_COMPARATIVE', 'L_ADV_SUPERLATIVE', 'PS_CONTRADICTION', 'PS_AGREEMENT', 'PS_EXAMPLES', 'PS_CONSEQUENCE', 'PS_CAUSE', 'PS_LOCATION', 'PS_TIME', 'PS_CONDITION', 'PS_MANNER', 'SY_QUESTION', 'SY_NARRATIVE', 'SY_NEGATIVE_QUESTIONS', 'SY_SPECIAL_QUESTIONS', 'SY_TAG_QUESTIONS', 'SY_GENERAL_QUESTIONS', 'SY_EXCLAMATION', 'SY_IMPERATIVE', 'SY_SUBORD_SENT', 'SY_SUBORD_SENT_PUNCT', 'SY_COORD_SENT', 'SY_COORD_SENT_PUNCT', 'SY_SIMPLE_SENT', 'SY_INVERSE_PATTERNS', 'SY_SIMILE', 'SY_FRONTING', 'SY_IRRITATION', 'SY_INTENSIFIER', 'SY_QUOT', 'VT_PRESENT_SIMPLE', 'VT_PRESENT_PROGRESSIVE', 'VT_PRESENT_PERFECT', 'VT_PRESENT_PERFECT_PROGR', 'VT_PRESENT_SIMPLE_PASSIVE', 'VT_PRESENT_PROGR_PASSIVE', 'VT_PRESENT_PERFECT_PASSIVE', 'VT_PAST_SIMPLE', 'VT_PAST_SIMPLE_BE', 'VT_PAST_PROGR', 'VT_PAST_PERFECT', 'VT_PAST_PERFECT_PROGR', 'VT_PAST_SIMPLE_PASSIVE', 'VT_PAST_POGR_PASSIVE', 'VT_PAST_PERFECT_PASSIVE', 'VT_FUTURE_SIMPLE', 'VT_FUTURE_PROGRESSIVE', 'VT_FUTURE_PERFECT', 'VT_FUTURE_PERFECT_PROGR', 'VT_FUTURE_SIMPLE_PASSIVE', 'VT_FUTURE_PROGR_PASSIVE', 'VT_FUTURE_PERFECT_PASSIVE', 'VT_WOULD', 'VT_WOULD_PASSIVE', 'VT_WOULD_PROGRESSIVE', 'VT_WOULD_PERFECT', 'VT_WOULD_PERFECT_PASSIVE', 'VT_SHOULD', 'VT_SHOULD_PASSIVE', 'VT_SHALL', 'VT_SHALL_PASSIVE', 'VT_SHOULD_PROGRESSIVE', 'VT_SHOULD_PERFECT', 'VT_SHOULD_PERFECT_PASSIVE', 'VT_MUST', 'VT_MUST_PASSIVE', 'VT_MUST_PROGRESSIVE', 'VT_MUST_PERFECT', 'VT_MST_PERFECT_PASSIVE', 'VT_CAN', 'VT_CAN_PASSIVE', 'VT_COULD', 'VT_COULD_PASSIVE', 'VT_CAN_PROGRESSIVE', 'VT_COULD_PROGRESSIVE', 'VT_COULD_PERFECT', 'VT_COULD_PERFECT_PASSIVE', 'VT_MAY', 'VT_MAY_PASSIVE', 'VT_MIGHT', 'VT_MIGHT_PASSIVE', 'VT_MAY_PROGRESSIVE', 'VT_MIGTH_PERFECT', 'VT_MIGHT_PERFECT_PASSIVE', 'VT_MAY_PERFECT_PASSIVE', 'ST_TYPE_TOKEN_RATIO_LEMMAS', 'ST_HERDAN_TTR', 'ST_MASS_TTR', 'ST_SENT_WRDSPERSENT', 'ST_SENT_DIFFERENCE', 'ST_REPETITIONS_WORDS', 'ST_REPETITIONS_SENT', 'ST_SENT_D_VP', 'ST_SENT_D_NP', 'ST_SENT_D_PP', 'ST_SENT_D_ADJP', 'ST_SENT_D_ADVP', 'L_I_PRON', 'L_HE_PRON', 'L_SHE_PRON', 'L_IT_PRON', 'L_YOU_PRON', 'L_WE_PRON', 'L_THEY_PRON', 'L_ME_PRON', 'L_YOU_OBJ_PRON', 'L_HIM_PRON', 'L_HER_OBJECT_PRON', 'L_IT_OBJECT_PRON', 'L_US_PRON', 'L_THEM_PRON', 'L_MY_PRON', 'L_YOUR_PRON', 'L_HIS_PRON', 'L_HER_PRON', 'L_ITS_PRON', 'L_OUR_PRON', 'L_THEIR_PRON', 'L_YOURS_PRON', 'L_THEIRS_PRON', 'L_HERS_PRON', 'L_OURS_PRON', 'L_MYSELF_PRON', 'L_YOURSELF_PRON', 'L_HIMSELF_PRON', 'L_HERSELF_PRON', 'L_ITSELF_PRON', 'L_OURSELVES_PRON', 'L_YOURSELVES_PRON', 'L_THEMSELVES_PRON', 'L_FIRST_PERSON_SING_PRON', 'L_SECOND_PERSON_PRON', 'L_THIRD_PERSON_SING_PRON', 'L_THIRD_PERSON_PLURAL_PRON', 'VF_INFINITIVE', 'G_PASSIVE', 'G_ACTIVE', 'G_PRESENT', 'G_PAST', 'G_FUTURE', 'G_MODALS_SIMPLE', 'G_MODALS_CONT', 'G_MODALS_PERFECT', 'AN', 'DDP', 'SVP', 'CDS', 'DDF', 'IS', 'PS', 'RE', 'ASF', 'ASM', 'OM', 'RCI', 'DMC', 'OR', 'QAS', 'PA', 'PR']

# trainCols = ['id', 'sentence_num', 'model', 'domain', 'POS_VERB', 'POS_NOUN', 'POS_ADJ', 'POS_ADV', 'POS_DET', 'POS_INTJ', 'POS_CONJ', 'POS_PART', 'POS_NUM', 'POS_PREP', 'POS_PRO', 'L_REF', 'L_HASHTAG', 'L_MENTION', 'L_RT', 'L_LINKS', 'L_CONT_A', 'L_FUNC_A', 'L_CONT_T', 'L_FUNC_T', 'L_PLURAL_NOUNS', 'L_SINGULAR_NOUNS', 'L_PROPER_NAME', 'L_PERSONAL_NAME', 'L_NOUN_PHRASES', 'L_PUNCT', 'L_PUNCT_DOT', 'L_PUNCT_COM', 'L_PUNCT_SEMC', 'L_PUNCT_COL', 'L_PUNCT_DASH', 'L_POSSESSIVES', 'L_ADJ_POSITIVE', 'L_ADJ_COMPARATIVE', 'L_ADJ_SUPERLATIVE', 'L_ADV_POSITIVE', 'L_ADV_COMPARATIVE', 'L_ADV_SUPERLATIVE', 'PS_CONTRADICTION', 'PS_AGREEMENT', 'PS_EXAMPLES', 'PS_CONSEQUENCE', 'PS_CAUSE', 'PS_LOCATION', 'PS_TIME', 'PS_CONDITION', 'PS_MANNER', 'SY_QUESTION', 'SY_NARRATIVE', 'SY_NEGATIVE_QUESTIONS', 'SY_SPECIAL_QUESTIONS', 'SY_TAG_QUESTIONS', 'SY_GENERAL_QUESTIONS', 'SY_EXCLAMATION', 'SY_IMPERATIVE', 'SY_SUBORD_SENT', 'SY_SUBORD_SENT_PUNCT', 'SY_COORD_SENT', 'SY_COORD_SENT_PUNCT', 'SY_SIMPLE_SENT', 'SY_INVERSE_PATTERNS', 'SY_SIMILE', 'SY_FRONTING', 'SY_IRRITATION', 'SY_INTENSIFIER', 'SY_QUOT', 'VT_PRESENT_SIMPLE', 'VT_PRESENT_PROGRESSIVE', 'VT_PRESENT_PERFECT', 'VT_PRESENT_PERFECT_PROGR', 'VT_PRESENT_SIMPLE_PASSIVE', 'VT_PRESENT_PROGR_PASSIVE', 'VT_PRESENT_PERFECT_PASSIVE', 'VT_PAST_SIMPLE', 'VT_PAST_SIMPLE_BE', 'VT_PAST_PROGR', 'VT_PAST_PERFECT', 'VT_PAST_PERFECT_PROGR', 'VT_PAST_SIMPLE_PASSIVE', 'VT_PAST_POGR_PASSIVE', 'VT_PAST_PERFECT_PASSIVE', 'VT_FUTURE_SIMPLE', 'VT_FUTURE_PROGRESSIVE', 'VT_FUTURE_PERFECT', 'VT_FUTURE_PERFECT_PROGR', 'VT_FUTURE_SIMPLE_PASSIVE', 'VT_FUTURE_PROGR_PASSIVE', 'VT_FUTURE_PERFECT_PASSIVE', 'VT_WOULD', 'VT_WOULD_PASSIVE', 'VT_WOULD_PROGRESSIVE', 'VT_WOULD_PERFECT', 'VT_WOULD_PERFECT_PASSIVE', 'VT_SHOULD', 'VT_SHOULD_PASSIVE', 'VT_SHALL', 'VT_SHALL_PASSIVE', 'VT_SHOULD_PROGRESSIVE', 'VT_SHOULD_PERFECT', 'VT_SHOULD_PERFECT_PASSIVE', 'VT_MUST', 'VT_MUST_PASSIVE', 'VT_MUST_PROGRESSIVE', 'VT_MUST_PERFECT', 'VT_MST_PERFECT_PASSIVE', 'VT_CAN', 'VT_CAN_PASSIVE', 'VT_COULD', 'VT_COULD_PASSIVE', 'VT_CAN_PROGRESSIVE', 'VT_COULD_PROGRESSIVE', 'VT_COULD_PERFECT', 'VT_COULD_PERFECT_PASSIVE', 'VT_MAY', 'VT_MAY_PASSIVE', 'VT_MIGHT', 'VT_MIGHT_PASSIVE', 'VT_MAY_PROGRESSIVE', 'VT_MIGTH_PERFECT', 'VT_MIGHT_PERFECT_PASSIVE', 'VT_MAY_PERFECT_PASSIVE', 'ST_TYPE_TOKEN_RATIO_LEMMAS', 'ST_HERDAN_TTR', 'ST_MASS_TTR', 'ST_SENT_WRDSPERSENT', 'ST_SENT_DIFFERENCE', 'ST_REPETITIONS_WORDS', 'ST_REPETITIONS_SENT', 'ST_SENT_D_VP', 'ST_SENT_D_NP', 'ST_SENT_D_PP', 'ST_SENT_D_ADJP', 'ST_SENT_D_ADVP', 'L_I_PRON', 'L_HE_PRON', 'L_SHE_PRON', 'L_IT_PRON', 'L_YOU_PRON', 'L_WE_PRON', 'L_THEY_PRON', 'L_ME_PRON', 'L_YOU_OBJ_PRON', 'L_HIM_PRON', 'L_HER_OBJECT_PRON', 'L_IT_OBJECT_PRON', 'L_US_PRON', 'L_THEM_PRON', 'L_MY_PRON', 'L_YOUR_PRON', 'L_HIS_PRON', 'L_HER_PRON', 'L_ITS_PRON', 'L_OUR_PRON', 'L_THEIR_PRON', 'L_YOURS_PRON', 'L_THEIRS_PRON', 'L_HERS_PRON', 'L_OURS_PRON', 'L_MYSELF_PRON', 'L_YOURSELF_PRON', 'L_HIMSELF_PRON', 'L_HERSELF_PRON', 'L_ITSELF_PRON', 'L_OURSELVES_PRON', 'L_YOURSELVES_PRON', 'L_THEMSELVES_PRON', 'L_FIRST_PERSON_SING_PRON', 'L_SECOND_PERSON_PRON', 'L_THIRD_PERSON_SING_PRON', 'L_THIRD_PERSON_PLURAL_PRON', 'VF_INFINITIVE', 'G_PASSIVE', 'G_ACTIVE', 'G_PRESENT', 'G_PAST', 'G_FUTURE', 'G_MODALS_SIMPLE', 'G_MODALS_CONT', 'G_MODALS_PERFECT', 'AN', 'DDP', 'SVP', 'CDS', 'DDF', 'IS', 'PS', 'RE', 'ASF', 'ASM', 'OM', 'RCI', 'DMC', 'OR', 'QAS', 'PA', 'PR']

train_df = train_df[trainCols]

train_df = train_df.rename(columns={
    'id_encoded': 'id',
})

train_df = train_df.sort_values(by=['id', 'sentence_num']).reset_index(drop=True)

In [33]:
# display(train_df.sample(5, random_state=42))
display(train_df.head())

,id,sentence_num,model,domain,POS_VERB,POS_NOUN,POS_ADJ,POS_ADV,POS_DET,POS_INTJ,...,RE,ASF,ASM,OM,RCI,DMC,OR,QAS,PA,PR
0,0,0,human,reviews,0.176471,0.117647,0.058824,0.058824,0.117647,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0,1,human,reviews,0.250000,0.125000,0.000000,0.250000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0,2,human,reviews,0.250000,0.125000,0.250000,0.125000,0.125000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.125000,0.0,0.0
3,0,3,human,reviews,0.230769,0.230769,0.153846,0.153846,0.153846,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.076923,0.0,0.0
4,0,4,human,reviews,0.333333,0.142857,0.047619,0.047619,0.095238,0.0,...,0.0,0.0,0.047619,0.0,0.0,0.0,0.0,0.095238,0.0,0.0


In [34]:
# Contar ids únicos por modelo
unique_ids_per_model = train_df.groupby('model')['id'].nunique()
print("Conteo de IDs únicos por modelo:")
print(unique_ids_per_model)
print("Total IDs únicos:", train_df['id'].nunique())

Conteo de IDs únicos por modelo:
model
chatgpt          35
cohere           31
cohere-chat      27
gpt2             64
gpt3             34
gpt4             24
human           500
llama-chat       50
mistral          54
mistral-chat     62
mpt              58
mpt-chat         61
Name: id, dtype: int64
Total IDs únicos: 1000


In [35]:
# Guardar el DataFrame final a un archivo CSV
pdtrainDF = pd.DataFrame(train_df)
pdtrainDF.to_csv('train_df.csv', index=False)